In [1]:
!pip install mlflow

In [15]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.124s


In [2]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path

import mlflow

In [3]:
experiment_id = mlflow.create_experiment(
    "MNIST MLP classifier", 
    artifact_location=Path.cwd().joinpath("./mlruns").as_uri(),
    tags={"version":"v1", "priority":"P1"}
)

In [4]:
experiment_id

'1'

In [5]:
mlflow.set_tracking_uri("sqlite:///mlruns/mlruns.db")

In [6]:
experiment = mlflow.set_experiment("MNIST MLP classifier")

2022/09/16 10:48:22 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/09/16 10:48:22 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [7]:
(train_images,train_labels),(test_images,test_labels) = mnist.load_data()
print(len(train_images))

11501568/11490434 [==============================] - 0s 0us/step
60000


In [32]:
model = keras.Sequential([layers.Dense(128,activation='relu'),
layers.Dense(10,activation='softmax')])
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [9]:
train_images = train_images.reshape((60000,28*28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000,28*28))
test_images = test_images.astype("float32") / 255

In [33]:
with mlflow.start_run(experiment_id=experiment_id, description="MLP model for classifying MNIST hand written digits", run_name="rmsprop-128N") as run:
  mlflow.keras.autolog(registered_model_name="MLP MNIST classifier")
  history = model.fit(train_images,train_labels,epochs = 5, batch_size=128)

2022/09/16 10:56:21 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.


Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: 0.3308 - accuracy: 0.9085
Epoch 2/5
469/469 [==============================] - 3s 7ms/step - loss: 0.1584 - accuracy: 0.9548
Epoch 3/5
469/469 [==============================] - 3s 7ms/step - loss: 0.1137 - accuracy: 0.9672
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0896 - accuracy: 0.9737
Epoch 5/5
469/469 [==============================] - 4s 8ms/step - loss: 0.0731 - accuracy: 0.9782


2022/09/16 10:56:38 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.8.2+zzzcolab20220719082949) contains a local version label (+zzzcolab20220719082949). MLflow logged a pip requirement for this package as 'tensorflow==2.8.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/09/16 10:56:44 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.8.2+zzzcolab20220719082949) contains a local version label (+zzzcolab20220719082949). MLflow logged a pip requirement for this package as 'tensorflow==2.8.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/09/16 10:56:44 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.3.15+cuda11.cudnn805) contains a local version label (+cuda11.cudnn805). ML

In [10]:
get_ipython().system_raw("mlflow server --host 0.0.0.0 -p 8000 --backend-store-uri sqlite:///mlruns/mlruns.db --default-artifact-root ./mlruns &")

In [17]:
get_ipython().system_raw('lt --port 8000 >> url.txt 2>&1 &')

In [16]:
# !rm url.txt

In [18]:
!cat url.txt

your url is: https://young-pugs-live-34-83-147-216.loca.lt


In [20]:
test_img = test_images[0:2][:]
test_labl = test_labels[0:2]
pred_labl = model.predict(test_img)

In [21]:
print(f"Predicted error {test_labl[0] - pred_labl[0].argmax()}") 

Predicted error 0


In [ ]:
test_loss, test_accuracy = model.evaluate(test_img,test_labl)
print('Test Loss: ', test_loss)
print('Test Accuracy: ',test_accuracy*100)

1/1 [==============================] - 0s 91ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Test Loss:  0.0
Test Accuracy:  100.0


In [ ]:
test_loss, test_accuracy = model.evaluate(test_images,test_labels)
print('Test Loss: ', test_loss)
print('Test Accuracy: ',test_accuracy*100)

313/313 [==============================] - 1s 3ms/step - loss: 0.1494 - accuracy: 0.9837
Test Loss:  0.14935661852359772
Test Accuracy:  98.36999773979187


## Loading model for predictions

In [22]:
run.info.run_id

'c77f7a6e37f04e68bc2c5fd6f2517bd1'

In [24]:
keras_model = mlflow.keras.load_model(f"runs:/{run.info.run_id}" + "/model")

In [25]:
predictions = keras_model.predict(test_img)

In [26]:
predictions[0].argmax()

7

In [27]:
test_images[0].shape

(784,)

## Infering with served model

In [ ]:
import requests
import json
import numpy as np

In [ ]:
json_data = {'data': np.expand_dims(test_images[0], axis=0).tolist()}

In [ ]:
headers = {'Content-type': 'application/json'}
res = requests.post('http://127.0.0.1:5000/invocations', headers=headers, data=json.dumps(json_data))
print(res.text)

[{"0": 4.766670946843021e-10, "1": 5.465215411303015e-13, "2": 9.226603481238271e-09, "3": 5.619980947813019e-05, "4": 3.853055600619776e-17, "5": 5.215898268851049e-10, "6": 6.385581625232239e-18, "7": 0.9999409914016724, "8": 6.213520808984185e-10, "9": 2.7281921575195156e-06}]
